In [1]:
import json
import os

import numpy as np
import spiceypy as spice
import pvl

In [2]:
kerneldir = '/work/projects/OSIRIS-REx/data/kernels/'

In [3]:
# Writer class to serialize numpy arrays
class NumpyAwareJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray) and obj.ndim == 1:
            lobj = obj.tolist()
            if len(lobj) == 1:
                return lobj[0]
            else:
                return lobj
        return json.JSONEncoder.default(self, obj)


# Utility Func for working with PVL
def find_in_dict(obj, key):
    """
    Recursively find an entry in a dictionary

    Parameters
    ----------
    obj : dict
          The dictionary to search
    key : str
          The key to find in the dictionary

    Returns
    -------
    item : obj
           The value from the dictionary
    """
    if key in obj:
        return obj[key]
    for k, v in obj.items():
        if isinstance(v,dict):
            item = find_in_dict(v, key)
            if item is not None:
                return item

In [4]:
isd= {}

#  Select the instrument

In [5]:
ikid = 64360  # PolyCam
#ikid = 64361  # MapCam
#ikid = 64362  # SamCam

In [14]:
opj = os.path.join
spice.furnsh(opj(kerneldir, 'new/bennu_v10.tpc'))
spice.furnsh(opj(kerneldir, 'new/FRAME.tf'))
spice.furnsh(opj(kerneldir, 'ik/orex_ocams_v02.ti'))
spice.furnsh(opj(kerneldir, 'iak/orex_ocams_addendum_v02.ti'))
spice.furnsh(opj(kerneldir, 'new/naif0011.tls'))
spice.furnsh(opj(kerneldir, 'sclk/SCLK.tsc'))
spice.furnsh(opj(kerneldir, 'new/de421.bsp'))

if ikid == 64360:
    # Total guess here...
    spice.furnsh(opj(kerneldir, 'ck/Phase03_AP_ShapeModel_5.bc'))
    spice.furnsh(opj(kerneldir, 'ck/Phase04_PS_PolyCam_1.bc'))
    spice.furnsh(opj(kerneldir, 'ck/Phase04_PS_PolyCam_2.bc'))
    spice.furnsh(opj(kerneldir, 'ck/Phase04_PS_PolyCam_3.bc'))
    spice.furnsh(opj(kerneldir, 'ck/Phase04_PS_PolyCam_4.bc'))
    spice.furnsh(opj(kerneldir, 'ck/Phase04_PS_PolyCam_5.bc'))
    spice.furnsh(opj(kerneldir, 'ck/Phase04_PS_PolyCam_6.bc'))

spice.furnsh(opj(kerneldir, 'new/orx_v03.tf'))
spice.furnsh(opj(kerneldir, 'tspk/sb-101955-76.bsp'))
spice.furnsh(opj(kerneldir, 'spk/OREX_DRMrevC_Phase03_AP_01s.bsp'))

In [7]:
# Load information from the IK kernel
isd['focal_length'] = spice.gdpool('INS-{}_FOCAL_LENGTH'.format(ikid), 0, 1)
isd['focal_length_epsilon'] = spice.gdpool('INS-{}_FL_UNCERTAINTY'.format(ikid), 0, 1)
isd['nlines'] = spice.gipool('INS-{}_PIXEL_LINES'.format(ikid), 0, 1)
isd['nsamples'] = spice.gipool('INS-{}_PIXEL_SAMPLES'.format(ikid), 0, 1)
isd['original_half_lines'] = isd['nlines'] / 2.0
isd['original_half_samples'] = isd['nsamples'] / 2.0
isd['ccd_center'] = spice.gdpool('INS-{}_CCD_CENTER'.format(ikid), 0, 2)
isd['ifov'] = spice.gdpool('INS-{}_IFOV'.format(ikid), 0, 1)
isd['boresight'] = spice.gdpool('INS-{}_BORESIGHT'.format(ikid), 0, 3)
isd['transx'] = spice.gdpool('INS-{}_TRANSX'.format(ikid), 0, 3)
isd['transy'] = spice.gdpool('INS-{}_TRANSY'.format(ikid), 0, 3)
isd['itrans_sample'] = spice.gdpool('INS-{}_ITRANSS'.format(ikid), 0, 3)
isd['itrans_line'] = spice.gdpool('INS-{}_ITRANSL'.format(ikid), 0, 3)
isd['focal_length'] = spice.gdpool('INS-{}_FOCAL_LENGTH'.format(ikid), 0, 1)

In [26]:
# Load the ISIS Cube header
header = pvl.load('/work/projects/OSIRIS-REx/ORIGINAL/A18111310005_PCAM_L2.cub')

isd['instrument_id'] = find_in_dict(header, 'InstrumentId')
isd['spacecraft_name'] = find_in_dict(header, 'SpacecraftName')
isd['target_name'] = find_in_dict(header, 'TargetName')

# Get the radii from SPICE
rad = spice.bodvrd(isd['target_name'], 'RADII', 3)
radii = rad[1]
isd['semi_a_axis'] = rad[1][0]
isd['semi_b_axis'] = rad[1][1]
isd['semi_c_axis'] = rad[1][2]

In [17]:
header

PVLModule([
  ('IsisCube',
   {'BandBin': PVLGroup([
    ('FilterName', 'Unknown')
  ]),
    'Core': {'Dimensions': {'Bands': 1,
                            'Lines': 1024,
                            'Samples': 1024},
             'Format': 'Tile',
             'Pixels': {'Base': 0.0,
                        'ByteOrder': 'Lsb',
                        'Multiplier': 1.0,
                        'Type': 'Real'},
             'StartByte': 65537,
             'TileLines': 512,
             'TileSamples': 512},
    'Instrument': {'ExposureDuration': 100,
                   'FocusPosition': 21510,
                   'InstrumentId': 'PolyCam',
                   'MissionName': 'OSIRIS-REx',
                   'SpacecraftClockStartCount': '1/0595381631.37945',
                   'SpacecraftName': 'OSIRIS-REX',
                   'StartTime': '2018 13 Nov 11:47:07.579',
                   'TargetName': 'Bennu'},
    'Kernels': PVLGroup([
    ('NaifFrameCode', -64360)
  ])})
  ('Label', PVLObjec

# Time

In [18]:
# Here convert the sclock
sclock = find_in_dict(header, 'SpacecraftClockStartCount')
exposure_duration = find_in_dict(header, 'ExposureDuration')
exposure_duration = exposure_duration * 0.001  # Scale to seconds

# Get the instrument id, and, since this is a framer, set the time to the middle of the exposure
spacecraft_id = spice.bods2c('OSIRIS-REX')
et = spice.scs2e(spacecraft_id, sclock)
et += (exposure_duration / 2.0)

isd['ephemeris_time'] = et

In [19]:
loc, _ = spice.spkpos(isd['target_name'], isd['ephemeris_time'], 'IAU_BENNU', 'LT+S', 'OSIRIS-REX')

# Scale to meters and reverse direction (since target=Mercury and observer=messenger)
# we want vector from Mercury origin to spacecraft position
isd['x_sensor_origin'] = loc[0] * -1000
isd['y_sensor_origin'] = loc[1] * -1000
isd['z_sensor_origin'] = loc[2] * -1000

# Sensor Orientation

In [20]:
camera2bodyfixed = spice.pxform('ORX_OCAMS_POLYCAM','IAU_BENNU', isd['ephemeris_time'])
opk = spice.m2eul(camera2bodyfixed, 3, 2, 1)

isd['omega'] = opk[2]
isd['phi'] = opk[1]
isd['kappa'] = opk[0]

# Sun Position

In [21]:
# Get the sun's position relative to a Mercury-fixed frame.
target = "SUN"
et = isd['ephemeris_time']
reference_frame = "IAU_BENNU"
light_time_correction = "LT+S"
observer = "BENNU"

sun_state, lt = spice.spkezr(target,
                             et,
                             reference_frame,
                             light_time_correction,
                             observer)

# Convert to meters
isd['x_sun_position'] = sun_state[0] * 1000
isd['y_sun_position'] = sun_state[1] * 1000
isd['z_sun_position'] = sun_state[2] * 1000
print("SUN POSITION (m): {} {} {}".format(sun_state[0]*1000,
                                          sun_state[1]*1000,
                                          sun_state[2]*1000))

SUN POSITION (m): -72584382155.74005 -131469457446.03712 -4308117323.711538


# Done!
<span style="font-size:48pt;">💵</span>

In [22]:
# Write it out
with open('A18111310005_PCAM_L2.isd', 'w') as f:
    f.write(json.dumps(isd, f, cls=NumpyAwareJSONEncoder, sort_keys=True, indent=4))

In [23]:
!cat A18111310005_PCAM_L2.isd

{
    "boresight": [
        0.0,
        0.0,
        1.0
    ],
    "ccd_center": [
        511.5,
        511.5
    ],
    "ephemeris_time": 595381695.8116988,
    "focal_length": 630.0,
    "focal_length_epsilon": 5.0,
    "ifov": 13.475999999999999,
    "instrument_id": "PolyCam",
    "itrans_line": [
        0.0,
        0.0,
        -117.64705882353
    ],
    "itrans_sample": [
        0.0,
        117.64705882353,
        0.0
    ],
    "kappa": -2.8464286672469763,
    "nlines": 1024,
    "nsamples": 1024,
    "omega": -1.5943142528879,
    "original_half_lines": 512.0,
    "original_half_samples": 512.0,
    "phi": 1.138936933411424,
    "semi_a_axis": 0.2825,
    "semi_b_axis": 0.2675,
    "semi_c_axis": 0.254,
    "spacecraft_name": "OSIRIS-REX",
    "target_name": "Bennu",
    "transx": [
        0.0,
        0.0085,
        0.0
    ],
    "transy": [
        0.0,
        0.0,
        -0.0085
    ],
    "x_sensor_origin": -23746.382688576374,
    "x_sun_position": -725843